In [25]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import img_to_array, load_img

In [26]:
# Load CSV files
train_csv_file = 'labels/classes_train.csv'
val_csv_file = 'labels/classes_valid.csv'
train_df = pd.read_csv(train_csv_file)
val_df = pd.read_csv(val_csv_file)

# Define parameters
train_image_dir = 'train/'
val_image_dir = 'valid/'
image_size = (224, 224)
num_classes = train_df.shape[1] - 1  # The number of classes (columns - 1 for filename)

def preprocess_image(directory, filename):
    img_path = os.path.join(directory, filename)
    img = load_img(img_path, target_size=image_size)
    img = img_to_array(img)
    img = img / 255.0  # Normalize to [0, 1]
    return img

def preprocess_label(row):
    return row[1:].values  # All columns except the filename

# Create lists for images and labels
train_images = []
train_labels = []
val_images = []
val_labels = []

for index, row in train_df.iterrows():
    filename = row['filename']
    label = preprocess_label(row)
    
    if os.path.exists(os.path.join(train_image_dir, filename)):
        image = preprocess_image(train_image_dir, filename)
        train_images.append(image)
        train_labels.append(label)

for index, row in val_df.iterrows():
    filename = row['filename']
    label = preprocess_label(row)
    
    if os.path.exists(os.path.join(val_image_dir, filename)):
        image = preprocess_image(val_image_dir, filename)
        val_images.append(image)
        val_labels.append(label)

# Convert lists to numpy arrays
train_images = np.array(train_images)
train_labels = np.array(train_labels)
val_images = np.array(val_images)
val_labels = np.array(val_labels)

In [27]:
train_images = np.asarray(train_images, dtype=np.float32)
train_labels = np.asarray(train_labels, dtype=np.int64)

val_images = np.asarray(val_images, dtype=np.float32)
val_labels = np.asarray(val_labels, dtype=np.int64)

In [28]:
batch_size = 32

# Create a TensorFlow dataset for training
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = train_dataset.shuffle(buffer_size=len(train_images))
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

# Create a TensorFlow dataset for validation
val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
val_dataset = val_dataset.batch(batch_size)
val_dataset = val_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)


In [30]:

# Load pre-trained VGG19 model + higher level layers
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers on top of VGG19
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(5, activation='sigmoid')(x)

# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of VGG19
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [31]:
# Train the model
epochs = 10

history = model.fit(
    train_dataset,
    epochs=epochs,
    validation_data=val_dataset
)


Epoch 1/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 19s 513ms/step - accuracy: 0.3809 - loss: 0.4921 - val_accuracy: 0.4646 - val_loss: 0.4455
Epoch 2/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 16s 498ms/step - accuracy: 0.5657 - loss: 0.3816 - val_accuracy: 0.5354 - val_loss: 0.4013
Epoch 3/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 16s 500ms/step - accuracy: 0.6335 - loss: 0.3360 - val_accuracy: 0.5556 - val_loss: 0.3958
Epoch 4/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 16s 500ms/step - accuracy: 0.6256 - loss: 0.3302 - val_accuracy: 0.5859 - val_loss: 0.3730
Epoch 5/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 16s 499ms/step - accuracy: 0.6845 - loss: 0.2950 - val_accuracy: 0.6061 - val_loss: 0.3467
Epoch 6/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 17s 506ms/step - accuracy: 0.7139 - loss: 0.2825 - val_accuracy: 0.5253 - val_loss: 0.3985
Epoch 7/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 17s 508ms/step - accuracy: 0.7109 - loss: 0.2692 - val_accuracy: 0.6566 - val_loss: 0.3499
Epoch 8/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 16s 499ms/step - accuracy: 0.7304 - loss: 0.2635 - val_accu

In [33]:
# Unfreeze some layers of VGG19 and re-train
for layer in base_model.layers[:15]:
    layer.trainable = False
for layer in base_model.layers[15:]:
    layer.trainable = True

# Re-compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

# Fine-tune the model
history_fine = model.fit(
    train_dataset,
    epochs=epochs,
    validation_data=val_dataset
)


Epoch 1/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 24s 695ms/step - accuracy: 0.7157 - loss: 0.2434 - val_accuracy: 0.6970 - val_loss: 0.3353
Epoch 2/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 22s 678ms/step - accuracy: 0.8432 - loss: 0.1695 - val_accuracy: 0.6768 - val_loss: 0.3349
Epoch 3/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 22s 677ms/step - accuracy: 0.8900 - loss: 0.1402 - val_accuracy: 0.6667 - val_loss: 0.3748
Epoch 4/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 22s 676ms/step - accuracy: 0.9195 - loss: 0.1013 - val_accuracy: 0.7172 - val_loss: 0.3673
Epoch 5/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 22s 676ms/step - accuracy: 0.9409 - loss: 0.0796 - val_accuracy: 0.7172 - val_loss: 0.3991
Epoch 6/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 22s 682ms/step - accuracy: 0.9540 - loss: 0.0596 - val_accuracy: 0.7172 - val_loss: 0.3883
Epoch 7/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 22s 675ms/step - accuracy: 0.9682 - loss: 0.0448 - val_accuracy: 0.7273 - val_loss: 0.4217
Epoch 8/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 22s 678ms/step - accuracy: 0.9499 - loss: 0.0363 - val_accu